<a href="https://colab.research.google.com/github/ric4234/AI-Fridays/blob/main/Analisi%20Di%20Immagini/01_NLP.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Object Detection with Transformers





The goal of this exercise is to build a model capable of understanding what a figure displays.

More specifically, Object Detection consist in detecting all objects in a image. Object Detection combines both classification and localization, because for each object you detect in an image you have also to provide label of the instance.

To do so, we will build a pipeline object from Transformers. Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs and also save the time and resources required to train a model from scratch.
At this link you can find more about Transformers: https://huggingface.co/docs/transformers/index.


#### 1 - Install dependencies and create utils functions

Firstly, we make sure to install all the needed libraries

In [ ]:
!pip install transformers
!pip install gradio
!pip install timm
!pip install inflect
!pip install phonemizer
!pip install timm

The following code contains few utils function that will be used in the nexts sections, like rendering the object detection result in an image or loading an image from URL

In [2]:
import io
import matplotlib.pyplot as plt
import requests
import inflect
from PIL import Image

def load_image_from_url(url):
    return Image.open(requests.get(url, stream=True).raw)

def render_results_in_image(in_pil_img, in_results):
    plt.figure(figsize=(16, 10))
    plt.imshow(in_pil_img)

    ax = plt.gca()

    for prediction in in_results:

        x, y = prediction['box']['xmin'], prediction['box']['ymin']
        w = prediction['box']['xmax'] - prediction['box']['xmin']
        h = prediction['box']['ymax'] - prediction['box']['ymin']

        ax.add_patch(plt.Rectangle((x, y),
                                   w,
                                   h,
                                   fill=False,
                                   color="green",
                                   linewidth=2))
        ax.text(
           x,
           y,
           f"{prediction['label']}: {round(prediction['score']*100, 1)}%",
           color='red'
        )

    plt.axis("off")

    # Save the modified image to a BytesIO object
    img_buf = io.BytesIO()
    plt.savefig(img_buf, format='png',
                bbox_inches='tight',
                pad_inches=0)
    img_buf.seek(0)
    modified_image = Image.open(img_buf)

    # Close the plot to prevent it from being displayed
    plt.close()

    return modified_image

def summarize_predictions_natural_language(predictions):
    summary = {}
    p = inflect.engine()

    for prediction in predictions:
        label = prediction['label']
        if label in summary:
            summary[label] += 1
        else:
            summary[label] = 1

    result_string = "In this image, there are "
    for i, (label, count) in enumerate(summary.items()):
        count_string = p.number_to_words(count)
        result_string += f"{count_string} {label}"
        if count > 1:
          result_string += "s"

        result_string += " "

        if i == len(summary) - 2:
          result_string += "and "

    # Remove the trailing comma and space
    result_string = result_string.rstrip(', ') + "."

    return result_string


##### To ignore warnings #####
import warnings
import logging
from transformers import logging as hf_logging

def ignore_warnings():
    # Ignore specific Python warnings
    warnings.filterwarnings("ignore", message="Some weights of the model checkpoint")
    warnings.filterwarnings("ignore", message="Could not find image processor class")
    warnings.filterwarnings("ignore", message="The `max_size` parameter is deprecated")

    # Adjust logging for libraries using the logging module
    logging.basicConfig(level=logging.ERROR)
    hf_logging.set_verbosity_error()

########

#### 2 - Build and use Object-Detection Pipeline

At this point, we create the object detection pipeline using a model from facebook, which is the detr-resnet-50. For more informations about this model you can see the documentation on Huggingface https://huggingface.co/facebook/detr-resnet-50-dc5.
You can find also a lot of other object detection model from Huggingface hub: https://huggingface.co/models?pipeline_tag=object-detection. To choose a model, you can use some metrics like the number of download, likes, or the evaulation metrics provided by the creator of the model

In [ ]:
from transformers import pipeline
od_pipe = pipeline("object-detection", "facebook/detr-resnet-50") # Add , device = "cuda:0" if this is running on a GPU

Then, we load the image and we use the pipeline object created below (od pipe) to detect objects in the choosen image. Objects will be saved in the ouput of the pipeline

In [ ]:
from PIL import Image
import requests
from io import BytesIO

# Fetch image from URL
url = 'https://decider.com/wp-content/uploads/2016/09/it-crowd-cult-corner.jpg'  # Replace with your image URL

raw_image = load_image_from_url(url)

raw_image

In [ ]:
pipeline_output = od_pipe(raw_image)

pipeline_output

In [ ]:
processed_image = render_results_in_image(
    raw_image,
    pipeline_output)
processed_image

Finally, we can translate the image in natural language using the summarize_predictions_natural_language method created in the previous section

In [ ]:
text = summarize_predictions_natural_language(pipeline_output)
text

#### 3 - Create a simple interface using Gradio

In this section, we will use Gradio to build a simple user interface (UI) where we can pass an image and then use the pipeline created below to get the result image.

Gradio interface expects a method where you pass the input (in this case an image) and return the output (the processed image), so the first thing that we will do is to create that method. You can explore more about Gradio on the official documentation: https://www.gradio.app/docs/interface

In [ ]:
import os
import gradio as gr

def get_pipeline_prediction(pil_image):

    pipeline_output = od_pipe(pil_image)

    processed_image = render_results_in_image(pil_image,
                                            pipeline_output)
    return processed_image

Once the method is created, we have to build a Gradio Interface object that will use that method

In [ ]:
demo = gr.Interface(
  fn=get_pipeline_prediction,
  inputs=gr.Image(label="Input image",
                  type="pil"), # Python Image Library Object
  outputs=gr.Image(label="Output image with predicted instances",
                   type="pil") # Python Image Library Object
)

Then we will launch the Interface. Launch the Interface means to create a simple web server endpoint that you can use to process your own images, directly or via API.



In [ ]:
demo.launch(share=True)

Remember to close you demo if you don't want to use it anymore

In [ ]:
demo.close()